In [65]:
class Expense_Tracker:
    def __init__(self):
        self.expenses = []

    def load_existing_data(self):
        try:
            self.expenses = [] 
            with open("expenses.txt", "r") as file:
                lines = file.readlines()

                if len(lines) <= 2:
                    print("No expenses to load.")
                    return

                # Print table headers
                print(f"{'Payer':<10} | {'Amount':<10} | {'Description':<25} | Receivers")
                print("-" * 70)

                for line in lines[2:]:  
                    parts = line.strip().split(" | ")
                    if len(parts) == 4:
                        payer = parts[0].strip()
                        amount = float(parts[1].strip())
                        description = parts[2].strip()
                        receiver_data = parts[3].strip().split(",")

                        receiver_dict = {}
                        receivers_list = []
                        for r in receiver_data:
                            if ": " in r:
                                name, val = r.strip().split(": ")
                                receiver_dict[name.strip()] = float(val.strip())
                                receivers_list.append(f"{name.strip()}: {val.strip()}")

                
                        self.expenses.append({
                            'payer': payer,
                            'amount': amount,
                            'description': description,
                            'receiver': receiver_dict
                        })

         
                        receivers_str = ", ".join(receivers_list)
                        print(f"{payer:<10} | {amount:<10.2f} | {description:<25} | {receivers_str}")

                print("\nExisting data loaded and displayed successfully.")

        except Exception as e:
            print(f"Error loading data: {e}")
    
    def add_expense(self):
        payer = input("Enter the payer name: ").strip()
        amt = float(input("Enter the amount: "))
        description = input("Enter the description of expense: ").strip()
        split_type = input('Choose the split type [Equal/Custom]: ').lower()
        receiver = {}

        if split_type == 'equal':
            receivers = [i.strip() for i in input("Enter the names of the receivers: ").split(",")]
            for i in receivers:
                receiver[i] = amt / len(receivers)

        elif split_type == 'custom':
            while True:
                receivers = [i.strip() for i in input("Enter the names of the receivers: ").split(",")]
                receiver = {}
                for i in receivers:
                    receiver[i] = float(input(f"Enter the amount for {i}: "))
                if sum(receiver.values()) != amt:
                    print('Entered amount is not matching')
                    print('Enter your details again\n')
                    continue
                break
        else:
            print('Choose a valid split_type')

        self.expenses.append({'payer': payer, 'amount': amt, 'description': description, 'receiver': receiver})
        print("Expense added successfully")

        self.save_data()  

    def get_balance(self):
        balances = {}
        for expense in self.expenses:
            payer = expense['payer']
            amt = expense['amount']
            for receiver, amount in expense['receiver'].items():
                if payer != receiver:
                    if receiver not in balances:
                        balances[receiver] = 0
                    if payer not in balances:
                        balances[payer] = 0
                    balances[receiver] -= amount
                    balances[payer] += amount
        return balances

    def get_simplified(self):
        balances = self.get_balance()
        if balances is None or not balances:
            print("No balances to simplify. Please add some expenses first.")
            return []

        simplified = []
        for person, balance in balances.items():
            if balance < 0:
                simplified.append((person, balance))
        return simplified

    def get_person_summary(self):
        person = input("Enter the name of the person to get the summary: ").strip()
        total_paid = 0
        total_received = 0
        total_spent_on_self = 0  
        
        for expense in self.expenses:
            if expense['payer'].strip() == person:  
                total_paid += expense['amount']
            for receiver in expense['receiver']:
                if receiver.strip() == person:
                    if expense['payer'].strip() != person:
                        total_received += expense['receiver'][receiver]
            if person in expense['receiver'] and expense['payer'].strip() == person:
                total_spent_on_self += expense['receiver'][person]

        print(f"\nSummary for {person}:")
        print(f"Total Paid      : {total_paid}")
        print(f"Total Received  : {total_received}")
        print(f"Total Spent on Self : {total_spent_on_self}")
    
    def get_expense_summary(self):
        summary = []
        for expense in self.expenses:
            summary.append({
                'payer': expense['payer'],
                'amount': expense['amount'],
                'description': expense['description'],
                'receivers': expense['receiver']
            })
        return summary

    def save_data(self):
        try:
            with open("expenses.txt", "w") as file:
                file.write(f"{'Payer':<15} | {'Amount':<10} | {'Description':<25} | Receivers\n")
                file.write("=" * 100 + "\n")
                for expense in self.expenses:
                    payer = expense['payer']
                    amount = float(expense['amount'])
                    description = expense.get('description', '')
                    receivers = expense['receiver']
                    receiver_info = ", ".join([f"{name}: {amt}" for name, amt in receivers.items()])
                    file.write(f"{payer:<15} | {amount:<10.2f} | {description:<25} | {receiver_info}\n")
            print("Data saved successfully in clean format.")
        except Exception as e:
            print(f"Error saving data: {e}")



In [67]:
a= Expense_Tracker()

In [83]:
a.add_expense()

Enter the payer name:  o
Enter the amount:  300
Enter the description of expense:  t
Choose the split type [Equal/Custom]:  equal
Enter the names of the receivers:  o,r,m


Expense added successfully
Data saved successfully in clean format.


In [85]:
for i in a.expenses:
    print ((i))

{'payer': 'r', 'amount': 300.0, 'description': 'f', 'receiver': {'o': 100.0, 'r': 100.0, 'm': 100.0}}
{'payer': 'o', 'amount': 300.0, 'description': 't', 'receiver': {'o': 100.0, 'r': 100.0, 'm': 100.0}}


In [87]:
a.get_simplified()

[('m', -200.0)]

In [47]:
a.save_data()

Data saved successfully in clean format.


In [97]:
a.load_existing_data()

Payer      | Amount     | Description               | Receivers
----------------------------------------------------------------------
r          | 300.00     | f                         | o: 100.0, r: 100.0, m: 100.0
o          | 300.00     | t                         | o: 100.0, r: 100.0, m: 100.0

Existing data loaded and displayed successfully.


In [89]:
a.get_expense_summary()

[{'payer': 'r',
  'amount': 300.0,
  'description': 'f',
  'receivers': {'o': 100.0, 'r': 100.0, 'm': 100.0}},
 {'payer': 'o',
  'amount': 300.0,
  'description': 't',
  'receivers': {'o': 100.0, 'r': 100.0, 'm': 100.0}}]

In [91]:
a.get_balance()

{'o': 100.0, 'r': 100.0, 'm': -200.0}

In [45]:
a.load_existing_data()

Payer      | Amount     | Description               | Receivers
----------------------------------------------------------------------
r          | 300.00     | f                         | r: 100.0, a: 100.0

Existing data loaded and displayed successfully.


In [95]:
a.get_person_summary()

Enter the name of the person to get the summary:  m



Summary for m:
Total Paid      : 0
Total Received  : 200.0
Total Spent on Self : 0
